1 + 3

https://mikeinnes.github.io/2020/07/29/mjolnir.html
Mjolnir

https://scala-lms.github.io/

MetaOcaml book
Strymonas
ASP

http://okmij.org/ftp/meta-programming/

http://okmij.org/ftp/meta-programming/implementations.html#meta-scheme metascheme. Thedifference between bracketing and quasiquotation
Is Spiral an example? Maybe not. FFTW ATLAS

The trick is a way of sending data from runtime to compile time


How to partially evaluate mini kanren?



macro hygiene video:
https://www.youtube.com/watch?v=JePBb9-ychE&ab_channel=TheJuliaProgrammingLanguage

In [5]:
macro inception(f,ex)
    return quote
        x = $ex
        y = $f(x)
        z = $f(y)
        x,y,z
        end
end

x = 100
@macroexpand @inception sin x

quote
    #= In[5]:3 =#
    var"#7#x" = var"#7#x"
    #= In[5]:4 =#
    var"#8#y" = Main.sin(var"#7#x")
    #= In[5]:5 =#
    var"#9#z" = Main.sin(var"#8#y")
    #= In[5]:6 =#
    (var"#7#x", var"#8#y", var"#9#z")
end

In [9]:
macro inception(f,ex)
    return quote
        x = $(esc(ex))
        y = $(esc(f))(x)
        z = $(esc(f))(y)
        x,y,z
        end
end

x = 100
@macroexpand @inception sin x

quote
    #= In[9]:3 =#
    var"#11#x" = x
    #= In[9]:4 =#
    var"#12#y" = sin(var"#11#x")
    #= In[9]:5 =#
    var"#13#z" = sin(var"#12#y")
    #= In[9]:6 =#
    (var"#11#x", var"#12#y", var"#13#z")
end

In [8]:

# Huh. So julia macros are hygieneic. It's a pass that happens upon macro return. And esc let's you take in stuff from the outside.
#
macro foo()
    quote x -> x end
end

@macroexpand @foo()

quote
    #= In[8]:2 =#
    var"#10#x"->begin
            #= In[8]:2 =#
            var"#10#x"
        end
end

In [10]:
@code_lowered 1 + sin(2)

CodeInfo(
1 ─ %1 = Base.promote(x, y)
│   %2 = Core._apply_iterate(Base.iterate, Base.:+, %1)
└──      return %2
)

In [1]:
1 + 3

4

In [39]:
x = 3
:( :( 1 + $x ) )
:( :( 1 + $( $(x)) ) )



:($(Expr(:quote, :(1 + $(Expr(:$, 4))))))

In [5]:
c = :(1 + 3)

:(1 + 3)

In [3]:
:($(1 + 2) * 10)

:(3 * 10)

In [6]:
:($c + 10)

:((1 + 3) + 10)

In [7]:
pow(x,n) = n == 0 ? 1 : x * pow(x,n-1)

pow (generic function with 1 method)

In [12]:
divrem(4, 2)

(2, 0)

In [17]:
square(x) = x * x
function pow(x,n)
  if n == 0 
        return 1
  end
  (d,r) = divrem(n,2)
  if r == 0
        return square(pow(x,d))
  else
        return x * pow(x,n - 1)
  end
end

pow (generic function with 1 method)

In [22]:
pow(4,3)

64

In [12]:
function spow(x,n)
  if n == 0 
        return 1
    elseif n == 1
        return x
    end
  (d,r) = divrem(n,2)
  if r == 0
        return :( square( $(spow(x,d)) ))
  else
        return :( $x * $(spow(x,n - 1) ))
  end
end

spow (generic function with 2 methods)

In [13]:
spow(n) = :(x -> $( spow(:x, n) ))

spow (generic function with 2 methods)

In [18]:
eval(spow(4))(3)

81

Julia is not hygienic :(
We could recover this via gensym




In [19]:
let x = 1
    x
end

1

In [ ]:
:(let x = 1
    $(let body = $x
                :()
                end)
    end)

In [22]:
function myfilter(b, x)
    m = length(b)
    o = Array{Float64}(undef, length(x))
    for i in 1:length(x)
        if i < m 
            o[i] = x[i]
        else
            sum = 0
            for k in 1:m
                sum = sum + b[k] * x[i-k+1] 
            end
            o[i] = sum
        end
    end
    return o
end

myfilter (generic function with 1 method)

In [23]:
myfilter([1 1 1], 1:7)

7-element Array{Float64,1}:
  1.0
  2.0
  6.0
  9.0
 12.0
 15.0
 18.0

In [29]:
function smyfilter(b, x)
    m = length(b)
    quote
        o = Array{Float64}(undef, length(x))
        for i in 1:length(x)
            if i < $m 
                o[i] = x[i]
            else
                $(begin
                sum = 0
                for k in 1:m
                    sum = :( $sum + $(b[k]) * x[i-$k+1]) 
                end
                 :(o[i] = $sum)
                end
                  )

            end
        end
        return o
    end
end

smyfilter (generic function with 1 method)

In [30]:
smyfilter([1 2 3], :x)

quote
    #= In[29]:4 =#
    o = Array{Float64}(undef, length(x))
    #= In[29]:5 =#
    for i = 1:length(x)
        #= In[29]:6 =#
        if i < 3
            #= In[29]:7 =#
            o[i] = x[i]
        else
            #= In[29]:9 =#
            o[i] = ((0 + 1 * x[(i - 1) + 1]) + 2 * x[(i - 2) + 1]) + 3 * x[(i - 3) + 1]
        end
    end
    #= In[29]:20 =#
    return o
end

In [ ]:
struct Alt
    l
    r
end

struct Star
    it
end

struct 



In [ ]:
function match_string(p,s)
    for i in 1:length(s)
        for j in 1:length(p)
            if p[j] != s[i+j-1]
                break
            end
            if j == length(p)
                return i
            end
        end
    end
end

function match_string(p,s)
    for i in 1:(length(s) - length(p))
        all(  [ a==b  for (a,b) in zip(p,s[i:end])]) && return i
    end
    return 0
end

function match_string(p,s)
    while i < (length(s) - length(p))
        i += 1
    end
end

function match_string(p,s,i)
    all(  [ a==b  for (a,b) in zip(p,s[i:end])])
end

function match_string(p,s, knownhead, i)


    
function selfmatch(p, i) # determine where to jump to
    p = p[1:i]
    for j in 1:i
        p
        end
    
end
    

In [ ]:

struct Init
    head::PatTerm1 # f(~x1, ~x2, ~x3) We want to explode out a pattern with fresh variables.
    next::Int64
end

struct Compare
    v1::Symbol # compare two variables heads?
    v2::Symbol
end

struct Bind
    v::Symbol
    f::PatTerm
    next::Int64
end



Prog = Array{Union{Check,Bind}}

struct PatTerm1
    head::Symbol
    args::Array{Symbol} # variables
end

while true
    
end


function interp(prog, e::Bind)
    t = e.v
    e.f.head != t.head && return false
    length(e.f.args) != length(t.args) && return false
    for (n,a) in enumerate(e.f.args)
        a = t.args[n]
    end
    interp( prog[e.next], e)
end

function sequence(es)
    acc = :()
    for e in es
        acc = quote
                $acc
                $e
                end
    end
end

function interp(prog, e::Bind)
    t = e.v
    quote
        $(e.f.head) != $t.head && return false
        $(length(e.f.args)) != length($t.args) && return false
        $(   
             sequence( [:( $a = $t.args[$n])  for (n,a) in enumerate(e.f.args)]
         )
        $(interp( prog, prog[e.next]))
    end
end



In [54]:
module Machine

struct PatTerm1
    head::Symbol
    args::Array{Symbol} # variables
end

struct Bind
    v::Symbol
    f::PatTerm1
end

struct Compare
    v1::Symbol # compare two variables heads?
    v2::Symbol
end

function sequence(es)
    acc = :()
    for e in es
        acc = quote
                $acc
                $e
                end
    end
    return acc
end

function interp(prog, e::Bind)
    t = e.v
    quote
        $(e.f.head) != $t.head && return false
        $(length(e.f.args)) != length($t.args) && return false
        $(   
             sequence( [:( $a = $t.args[$n])  for (n,a) in enumerate(e.f.args)])
         )
        # $(interp( prog[2:end], prog[1]))
    end
end

function interp(prog, e::Compare)
    quote
        $(e.v1).head != $(e.v2).head && return false
        # $(interp( prog[2:end], prog[1]))
    end
end

f = Bind(:x, PatTerm1(:f, [:y,:z]))

display(interp([] ,f ))

end

quote
    #= In[54]:32 =#
    f != x.head && return false
    #= In[54]:33 =#
    2 != length(x.args) && return false
    #= In[54]:34 =#
    begin
        #= In[54]:22 =#
        begin
            #= In[54]:22 =#
            ()
            #= In[54]:23 =#
            y = x.args[1]
        end
        #= In[54]:23 =#
        z = x.args[2]
    end
end

Main.Machine

In [ ]:
function compile(v, x::PatVar, env)
    if haskey(env, x)
        [Check(v, env[x])]
    else
        env[x] = v
    end
end

function compile(v, x::PatTerm, env)
    # v = gensym(x.head)
    f = PatTerm1(x.head, [gensym() for i in x.])
    Bind(v, f)
    # or here we can do some optimizations. Such as try to do early stoppage / pruning
    for y, v in zip(x.args, f.args)
        prog = vcat(prog, compile(v, y, env))
    end
    return prog
end

function compile(x::PatTerm)
    f = PatTerm1(x.head, [gensym() for i in x.args])
    prog = [Init(f)]
    for y, v in zip(x.args, f.args)
        prog = vcat(prog, compile(v, y, env))
    end
    prog = vcat(prog, Yield(env))
    return prog # implicilty we're also returning env?
end

# yeah, so obviously we're bvasically doing some kind of traversal of the pattern. Linearizing it.
# 
    

    

In [46]:
dump(:(
        begin
    a = 1
    b = 2
        end
    ))

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol In[46]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol a
        2: Int64 1
    3: LineNumberNode
      line: Int64 4
      file: Symbol In[46]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol b
        2: Int64 2
